# Import libraries

In [1]:
# ! pip install faker 

In [32]:
import pandas as pd
import numpy as np
import random
from faker import Faker
from datetime import datetime, timedelta
from uuid import uuid4
from tqdm import tqdm
from random import randint, uniform
from faker.providers import BaseProvider

In [3]:
fake = Faker()
start_date = datetime.strptime("2005-01-01", "%Y-%m-%d")
end_date = datetime.strptime("2023-12-31", "%Y-%m-%d")

In [4]:
def generate_random_id():
    return(uuid4().hex)

# Data generation

## Customer data

In [5]:
from random import randint, uniform
from faker.providers import BaseProvider

class CameroonPhoneProvider(BaseProvider):
    def phone_number(self):
        """Generates a fake Cameroon phone number in a random format."""
        formats = [
            "+237 65ABCDEFG",
            "+237 67ABCDEFG",
            "+237 68ABCDEFG",
            "+237 69ABCDEFG",
        ]
        format_str = formats[randint(0, len(formats) - 1)]
        phone_number = format_str.replace("A", str(randint(0, 9)))
        phone_number = phone_number.replace("B", str(randint(0, 9)))
        phone_number = phone_number.replace("C", str(randint(0, 9)))
        phone_number = phone_number.replace("D", str(randint(0, 9)))
        phone_number = phone_number.replace("E", str(randint(0, 9)))
        phone_number = phone_number.replace("F", str(randint(0, 9)))
        phone_number = phone_number.replace("G", str(randint(0, 9)))
        return phone_number

fake = Faker()
fake.add_provider(CameroonPhoneProvider)

cam_phone_number = fake.phone_number()
print(cam_phone_number)

+237 672264599


In [6]:
customers_list = []
for i in tqdm(range(10000)):
    first_name = fake.first_name()
    last_name = fake.last_name()
    domain = fake.domain_name()
    email = f"{first_name}.{last_name}@{domain}"
    phone_number = cam_phone_number
    registration_date = fake.date_between(start_date, end_date)
    date_of_birth = fake.date_of_birth(minimum_age=30, maximum_age=75)
    address = fake.address()
    city = random.choice(["Yaoundé", "Douala", "Bafoussam", "Dschang", "Garoua", "Maroua", "Bamenda", "Ngaoundéré", "Nkongsamba", "Limbe", "Bertoua", "Mbouda", "Guider","Foumban", "Tibati", "Buea", "Kumba", "Kribi", "Loum", "Nkambe","Bangangté", "Baham", "Ndikiniméki", "Wum", "Batouri", "Yokadouma", "Mindif", "Meigang", "Abong-Mbang", "Bertoua", "Bogo", "Dir", "Poli", "Mbouda", "Bakoum", "Ngaoundal", "Figuil", "Mayo-Darlé", "Mayo-Rey", "Betaré-Oya", "Guidiguis", "Minjibir", "Mbankomo", "Ntui", "Mengang", "Akonolinga", "Ebolowa", "Abé", "Sangmélima", "Oveng", "Bipindi", "Meyomessé",])
    region = random.choice(["Nord", "Sud", "Est", "Ouest", "Centre", "Littoral", "Adamaoua", "Nord-ouest", "Sud-ouest", "Extrème-nord"])

    date_dict = {
        'customer_id' :generate_random_id(), 
        'first_name': first_name, 
        'last_name': last_name,
        'email': email,
        'phone_number': phone_number,
        'registration_date': registration_date,
        'date_of_birth' : date_of_birth,
        'address': address,
        'city': city,
        'region' : region
        }
    customers_list.append(date_dict)
customer_df = pd.DataFrame(customers_list)
print(customer_df.shape)
customer_df.head()

100%|██████████| 10000/10000 [00:20<00:00, 499.03it/s]


(10000, 10)


,customer_id,first_name,last_name,email,phone_number,registration_date,date_of_birth,address,city,region
0,25c8d65df6904638996b3882aaa6158e,Michael,Martin,Michael.Martin@robinson-zimmerman.biz,+237 672264599,2021-04-25,1981-07-02,Unit 7769 Box 3583\nDPO AE 29804,Mengang,Nord-ouest
1,db855153896f4de6bc462edae36a31e0,Robert,Petty,Robert.Petty@phillips.net,+237 672264599,2018-12-31,1966-12-26,"19977 Martinez Wall\nLake Devin, AK 98409",Yaoundé,Nord
2,5005ad3156f845128c1e3ed12f4cf81c,Melinda,Cox,Melinda.Cox@hall.com,+237 672264599,2008-02-28,1966-04-26,"379 White Manor\nPort Candicechester, TX 30544",Garoua,Ouest
3,1040b9a8c5294c49897e27a47a4b9b4c,Ashley,Clayton,Ashley.Clayton@burns.net,+237 672264599,2021-10-21,1990-09-06,"97526 Michael Island Apt. 998\nMariafort, NH 8...",Nkongsamba,Sud
4,c3b7293de4be49b3a61c06442642c35a,Brandon,Gordon,Brandon.Gordon@dodson.com,+237 672264599,2008-05-01,1982-07-28,"64053 Walsh Track\nBakerland, NV 62257",Wum,Extrème-nord


In [26]:
customer_df.to_csv('../Data/Train/customers.csv', index=False)

## Device data

In [11]:

#creation de id

def create_id():
    return str(uuid4())

#creation des types d'appareils
def generate_device_type():
    return random.choice(['mobile', 'tablette'])

#creation des os

def generate_os():
    return random.choice(['android', 'ios', 'mac', 'windows', 'linux'])

#creation des fabricants
manufacturer_and_model={
                        'dell':['XPS 13', 'INSPIRON 15', 'LATITUDE 7410', 'PRECISION 5550'],
                        'HP': ['spectre x360', 'envy 13', 'pavillon 15', 'OMEN 15'],
                        'LG': ['spectre x3670', 'envy45','Gram 17', 'Wing 5G', ' NanoCell TV'],
                        'SONY':['Xperia 1 III', 'Xperia 5 II', 'Bravia OLDE', 'VAIO SX14'],
                        'Apple':['iphone 13', 'ipad pro', 'iphone Xr'],
                        'Sharp':['Aquos R6', 'aquos sense 5g', ' dynabook v8']
                        }
#def generate_manufacturer():
    #return random.choices(['dell', 'hp', 'lg', 'sony', 'apple', 'sharp', 'lenovo', 'acer', 'tecno','itel', 'huawei', 'samsung', 'google'])

#creation des modeles

def generate_model(type):
    return random.choice(manufacturer_and_model[type])

#creation de la table devices

def create_devices_table(lenght= 100000):

    devices= pd.DataFrame()

    # les listes
    device_id_list=[]
    devices_type_list=[]
    devices_manufacturer_list=[]
    devices_model_list=[]
    devices_last_used_list=[]
    devices_os_list=[]

    #generer les valeurs
    for i in tqdm(range(lenght)):
        devices_id=create_id()
        devices_type= generate_device_type()
        devices_last_used= fake.date_time_between(start_date= '-5y', end_date='now')
        devices_os= generate_os()
        devices_manufacturer= random.choice(list(manufacturer_and_model.keys()))
        devices_model= generate_model(devices_manufacturer)

        #ajout dans les listes
        device_id_list.append(devices_id)
        devices_type_list.append(devices_type)
        devices_model_list.append( devices_model)
        devices_manufacturer_list.append( devices_manufacturer)
        devices_last_used_list.append(devices_last_used)
        devices_os_list.append(devices_os)
        
     #assigner aux colonnes de la table devices
    devices['device_id']= device_id_list
    devices['device_type']= devices_type_list
    devices['os']= devices_os_list
    devices['manufacturer']= devices_manufacturer_list
    devices['model']= devices_model_list
    devices['last_used']= devices_last_used_list

    return devices

devices= create_devices_table()
devices

100%|██████████| 100000/100000 [00:09<00:00, 10884.98it/s]


,device_id,device_type,os,manufacturer,model,last_used
0,2bbc4d13-af28-4bcf-a704-71c954c37e53,tablette,windows,HP,OMEN 15,2023-10-18 17:28:27
1,9abcf743-c4d6-4961-979b-128f51f9220d,mobile,windows,LG,NanoCell TV,2023-02-20 19:53:58
2,992c1261-0216-4333-ab81-e77715579b21,mobile,windows,LG,Wing 5G,2021-06-12 13:19:46
3,1ca1dfa6-c318-45ae-a7f7-02596aecb78c,mobile,android,Apple,iphone Xr,2020-08-13 11:05:56
4,b3e0ca71-2212-48b2-a601-ce37dfc574e1,mobile,ios,Sharp,Aquos R6,2024-01-03 10:13:59
...,...,...,...,...,...,...
99995,3c3db361-85e7-4e23-983e-b76985aa790a,mobile,windows,LG,NanoCell TV,2020-05-19 11:09:28
99996,f5e69f8d-cc0d-465a-bae7-5b97c61b1b71,tablette,android,SONY,Xperia 1 III,2024-01-07 23:32:04
99997,fea87837-b90f-4d7b-b465-0ff2663d1de4,mobile,ios,dell,LATITUDE 7410,2020-07-25 23:21:23
99998,370913e6-6211-4359-8f12-ae2f927eb36b,mobile,mac,HP,envy 13,2023-04-05 06:29:16


In [27]:
devices.to_csv('../Data/Train/devices.csv', index=False)

## Transaction data

In [36]:
location_list = []
for i in tqdm(range(1000)):
    location = fake.address()
    location_list.append(location)

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:01<00:00, 852.69it/s]


In [37]:
transaction_list=[]
customer_id_list = customer_df["customer_id"]

for i in tqdm(range(100000)):
    transaction_id = generate_random_id()
    customer_id = random.choice(customer_id_list)
    transaction_type = random.choice(["retrait", "depôt", "transfert"])
    amount = uniform(100, 10000000)
    transaction_date = fake.date_between(start_date, end_date)
    location = random.choice(location_list)
    device_id = random.choice(devices["device_id"])
    status = random.choice(["échouée", "réussie"])
    is_fraud = random.choice([0,1])

    transaction_dict = {
        "transaction_id": transaction_id,
        "customer_id": customer_id,
        "transaction_type": transaction_type,
        "amount": amount,
        "transaction_date": transaction_date,
        "location": location,
        "device_id": device_id,
        "status": status,
        "is_fraud": is_fraud
    }

    transaction_list.append(transaction_dict)

transaction_df = pd.DataFrame(transaction_list)
print(transaction_df.shape)
transaction_df.head()

100%|██████████| 100000/100000 [00:22<00:00, 4359.04it/s]


(100000, 9)


,transaction_id,customer_id,transaction_type,amount,transaction_date,location,device_id,status,is_fraud
0,2251f220c98e4da1adf6b5997b5c5391,bbafa72861294cfb96dfed3aa956bfbf,depôt,8.617840e+05,2010-02-25,"319 Galloway Dale Apt. 613\nNew Kevinbury, IL ...",1d843fad-0117-41bc-8ff2-208cffbac314,échouée,0
1,cb3f318268c246a3b91c96347782aa4f,218ad29d48bf41c3a15aacd26ee19ac0,transfert,4.160409e+06,2007-12-27,"9676 Hoffman Isle Apt. 254\nDaniellechester, H...",a28baef9-1e33-490b-88a7-1b07c54ea485,réussie,0
2,535332f5dd334dc98d85a93bacb76f8c,8787ee3f44624cb7a4569f75c6cefac8,retrait,5.741303e+06,2007-05-17,"096 Christopher Streets\nLake Joshua, CO 60898",a69bbae7-0055-40b5-beff-4b0457f7b52a,réussie,0
3,167733ffec654549bc2f9477f8157b3d,ea0b6e39a32e44028a0e484dd7d5aaa6,retrait,7.992655e+06,2015-08-14,"5496 Donald Roads\nSouth Courtneystad, WY 51100",6ea33d0a-c3cc-411f-bb65-c0c38512867b,réussie,0
4,0b5d7bdeebf5417fb0c3cf54b3825a66,85e37d1bd10641e7b56df0f9fa870fa4,transfert,3.606987e+06,2008-08-30,66933 Smith Grove Suite 162\nNew Stephaniemout...,d83a649c-41ea-4ed3-8038-38193c6308fe,échouée,0


In [38]:
transaction_df.to_csv('../Data/Train/transactions.csv', index=False)

## Transaction history data

In [16]:
transaction_history_list = []

for i, transaction in tqdm(transaction_df.iterrows()):
    history_id = generate_random_id()
    customer_id = transaction.customer_id
    transaction_id = transaction.transaction_id
    previous_balance = transaction.amount + uniform(0, 10000000)

    if transaction.transaction_type == "depôt":
        new_balance = previous_balance + transaction.amount
    else:
        new_balance = previous_balance - transaction.amount
        
    transaction_date = transaction.transaction_date

    transaction_history_dict = {
        "history_id" : history_id,
        "customer_id" : customer_id,
        "transaction_id" : transaction_id,
        "previous_balance" : previous_balance,
        "new_balance" : new_balance,
        "transaction_date": transaction_date
    }
    transaction_history_list.append(transaction_history_dict)

transaction_history_df = pd.DataFrame(transaction_history_list)
print(transaction_history_df.shape)
transaction_history_df.head()

100000it [00:26, 3805.05it/s]


(100000, 6)


,history_id,customer_id,transaction_id,previous_balance,new_balance,transaction_date
0,8b8f36c7ff314edd8a91e57316d20376,03ddb1c909d9487082e39760a6ad09cd,779370db6de54bde81cb40bdcc7d82dd,3.484714e+06,3.441105e+06,2020-08-10
1,85d77db69e404981962464f3e017867a,aab6e62f95884eb198d8b9a91b61d575,a912e208b3014184b790fa98cde4af7e,1.087710e+07,6.805382e+06,2011-01-03
2,e130ec7a967c4dbfb94525ca7f124dfa,90594f0a9a1f46db9c4db081fecf115b,f433d12901c848ad8c5edbb2e7fa4c64,6.548026e+06,5.570384e+06,2007-04-24
3,d60d7a603b7949d8ba433a4889f026b1,c27fb44b1d784d25b9e95c1258fb3428,05dd056503eb49609ee23b427c43ae6d,3.906832e+06,1.293205e+06,2008-10-26
4,6aebca6d73774771a0cf0f71d23c81e9,80528e1d1a3649bcb1d768630736dd3f,7eb841303d4a4a65b5a4c72cd92b16c1,6.228226e+06,3.901130e+06,2021-08-20


In [29]:
transaction_history_df.to_csv('../Data/Train/transaction_history.csv', index=False)

## Transaction pattern data

In [18]:
customer_id_list = customer_df["customer_id"]
print(customer_id_list)

0       25c8d65df6904638996b3882aaa6158e
1       db855153896f4de6bc462edae36a31e0
2       5005ad3156f845128c1e3ed12f4cf81c
3       1040b9a8c5294c49897e27a47a4b9b4c
4       c3b7293de4be49b3a61c06442642c35a
                      ...               
9995    78bd73024317420e8d2fefc44d93be77
9996    dd5898f9fb934800af03952c94c114aa
9997    ff196fdeda134e24838096e17fb04b8c
9998    0ac7a332afb847a3a4f536151b588290
9999    f8ac4d6c03564a6ca524bdcaa4aac174
Name: customer_id, Length: 10000, dtype: object


In [19]:
import numpy as np
import pandas as  pd 
import random
from faker import Faker
from uuid import uuid4
from tqdm import tqdm

faker= Faker()

# Fonction pour générer un identifiant unique
def create_id():
    return str(uuid4())

# Fonction pour générer un montant moyen aléatoire
def generate_average_amount():
    return round(random.uniform(5.0, 1000.0), 2)

# Fonction pour générer une fréquence de transactions aléatoire
def generate_frequency():
    return random.randint(1, 30)

# Fonction pour générer une localisation fréquente aléatoire
def generate_most_common_location():
    return faker.city()

# Fonction pour générer un appareil le plus utilisé aléatoire
def generate_most_common_device():
    return random.choice(['mobile', 'tablette'])

# Fonction pour créer une table de transaction patterns
def create_transaction_patterns_table(length=100000):

    transaction_patterns = pd.DataFrame()

    # Les listes
    pattern_id_list = []
    customer_id_list = customer_df["customer_id"].tolist()
    average_amount_list = []
    frequency_list = []
    most_common_location_list = []
    most_common_device_list = []
    foreign_key_list=[]  # liste des cles etrangeres qui seront aleatoirement choisies

    # Générer les valeurs
    for i in tqdm(range(length)):
        pattern_id = create_id()
        customer_id = random.choice(customer_id_list) 
        average_amount = generate_average_amount()
        frequency = generate_frequency()
        most_common_location = generate_most_common_location()
        most_common_device = generate_most_common_device()

        # Ajouter dans les listes
        pattern_id_list.append(pattern_id)
        foreign_key_list.append(customer_id)
        average_amount_list.append(average_amount)
        frequency_list.append(frequency)
        most_common_location_list.append(most_common_location)
        most_common_device_list.append(most_common_device)
    
    # Assigner aux colonnes de la table transaction patterns (en dehors de la boucle)
    transaction_patterns['pattern_id'] = pattern_id_list
    transaction_patterns['customer_id'] = foreign_key_list
    transaction_patterns['average_amount'] = average_amount_list
    transaction_patterns['frequency'] = frequency_list
    transaction_patterns['most_common_location'] = most_common_location_list
    transaction_patterns['most_common_device'] = most_common_device_list

    return transaction_patterns

# Créer une table de transaction patterns
transaction_patterns = create_transaction_patterns_table()

# Afficher les premières lignes de la table transaction patterns
transaction_patterns


100%|██████████| 100000/100000 [00:29<00:00, 3367.55it/s]


,pattern_id,customer_id,average_amount,frequency,most_common_location,most_common_device
0,6da198eb-e8fe-48df-be98-3e0b14ac182b,aedf7cb0313b40318b60507bf8ef9dd7,422.83,24,Karistad,mobile
1,6f4fd0e5-f23a-4498-841d-ff946d747a87,6b254399f4cf4b0c983bdeb95c18a981,439.71,27,New Dana,tablette
2,32e9d486-64f4-48be-ad56-3dace7059343,c8ceab85e02e46d894e41b9e42fffd9c,60.24,2,Thomasshire,mobile
3,6dfe347a-20e1-425b-925c-e13d24de5781,3f6a34021fa6428d9e439e0e856fd5b4,61.38,28,West Melissaport,tablette
4,b58d3fb8-1a1c-4513-92f7-937e1af6361a,6c4d9cd952b24cd58fc4d58431ad0957,565.41,28,Zacharymouth,tablette
...,...,...,...,...,...,...
99995,5d2de4a1-72de-4eac-bb72-4ff5e5472f28,f2ab6223bcc14acc83f299168d4666f2,704.49,14,Port Amystad,tablette
99996,a55d7983-fcea-41a6-a049-5ea23522402d,f28fce4c60ae45dabd5257274a21864f,598.87,3,Port Shawnmouth,tablette
99997,2cdcffe5-7e76-4d31-a709-78c72d6a578c,38ac1e7119e7470bafbe0c849b3529a0,944.82,13,South Amandafurt,mobile
99998,c5bd81c2-525e-47b1-9c3d-e0605b8c9221,6362a524f3f34d0dac04e41ca9197a7a,725.51,15,Lake Monicaborough,mobile


In [30]:
transaction_patterns.to_csv('../Data/Train/transaction_patterns.csv', index=False)

## alerts data

In [21]:
# Fonction pour générer un identifiant unique
def create_id():
    return str(uuid4())

# Fonction pour générer une date et heure aléatoire
def generate_alert_date():
    return faker.date_time_between(start_date='-5y', end_date='now')

# Fonction pour générer un type d'alerte aléatoire
def generate_alert_type():
    return random.choice(['suspicious activity', 'large transaction'])

# Fonction pour générer un statut d'alerte aléatoire
def generate_alert_status():
    return random.choice(['open', 'closed'])

# Fonction pour générer une description d'alerte aléatoire
def generate_description():
    return faker.text(max_nb_chars=250)

# Fonction pour créer une table d'alertes
def create_alerts_table(length=100):

    alerts = pd.DataFrame()

    # Les listes
    alert_id_list = []
    transaction_id_list = transaction_df["customer_id"].tolist()
    alert_date_list = []
    alert_type_list = []
    alert_status_list = []
    description_list = []
    foreign_key_alerts_list=[]

    # Générer les valeurs
    for i in tqdm(range(length)):
        alert_id = create_id()
        transaction_id = random.choice(transaction_id_list) # Remplacer par un identifiant de transaction réel si disponible
        alert_date = generate_alert_date()
        alert_type = generate_alert_type()
        alert_status = generate_alert_status()
        description = generate_description()

        # Ajouter dans les listes
        alert_id_list.append(alert_id)
        foreign_key_alerts_list.append(transaction_id)
        alert_date_list.append(alert_date)
        alert_type_list.append(alert_type)
        alert_status_list.append(alert_status)
        description_list.append(description)
    
    # Assigner aux colonnes de la table alerts (en dehors de la boucle)
    alerts['alert_id'] = alert_id_list
    alerts['transaction_id'] = foreign_key_alerts_list
    alerts['alert_date'] = alert_date_list
    alerts['alert_type'] = alert_type_list
    alerts['alert_status'] = alert_status_list
    alerts['description'] = description_list

    return alerts

# Créer une table d'alertes
alerts = create_alerts_table()

# Afficher les premières lignes de la table alerts
alerts

100%|██████████| 100/100 [00:00<00:00, 1011.21it/s]


,alert_id,transaction_id,alert_date,alert_type,alert_status,description
0,63b50a2e-a204-46a3-a350-d09cf0e01581,87d9ad903c6d47acadf744f06091dd38,2020-06-06 16:55:47,suspicious activity,closed,Similar treatment office plant. Tv produce suf...
1,8fa0e3f0-d8a5-45b5-9623-06d62044be70,51e070279b7d4803847c493ba75291c8,2024-05-02 00:16:43,suspicious activity,open,Because front eight kitchen apply true tonight...
2,a09d06d6-77a5-4e8c-87bd-2bcd02d05caf,70bea40e9525401a9ba60014dfbfc802,2023-12-12 16:26:39,large transaction,closed,Week deep better man second consumer some. Inc...
3,d5889251-8f62-4e74-927a-e72625fc5358,d40ef7dde4db4fcc937cac17748ee063,2021-09-07 11:33:07,large transaction,open,Create same able easy strong. Force environmen...
4,000af4a7-ec27-436d-a7c4-9d5e3b0559f1,306695e4937644fa952629f039f6404f,2023-10-06 01:15:39,suspicious activity,closed,From month doctor successful claim above perso...
...,...,...,...,...,...,...
95,f054e9c5-ef26-4a6e-926e-9fad84898c4f,ab37d5c6b7914f5ba8d062e3474ee733,2024-06-01 23:45:00,suspicious activity,open,Yeah wait bank.\nRoad next above together supp...
96,1e74fbfd-382c-41c1-8238-245220b47a77,bc7bd42470e94e0b98e081b71b87684e,2021-03-15 00:29:14,suspicious activity,open,Growth agency against expert then reality. Eve...
97,2743a767-8f3a-4fd9-87bd-f52e2de10b7a,ca46da07d43a40fcae4e1e6a5eb859ce,2024-06-08 16:17:22,suspicious activity,closed,Attention perform picture system act deep hope...
98,53395214-d4a5-48a4-9002-21484def7a8d,20f7383b9a89461ebd08ab50752ba719,2024-03-30 12:46:55,large transaction,open,Across key car baby everybody much program. So...


In [31]:
alerts.to_csv('../Data/Train/alerts.csv', index=False)

## fraude_cases data 

## regions data

In [23]:
from glob import glob

list_data = glob('../Data/Train/*')
print(list_data)


['../Data/Train\\alerts.csv', '../Data/Train\\customers.csv', '../Data/Train\\devices.csv', '../Data/Train\\transactions.csv', '../Data/Train\\transaction_history.csv', '../Data/Train\\transaction_patterns.csv']


In [24]:
import os 
os.sep

'\\'

In [25]:
list_data[0].split(os.sep)[-1].split('.')[0]

'alerts'